# Character Sequence to Sequence 
In this notebook, we'll build a model that takes in a sequence of letters, and outputs a sorted version of that sequence. We'll do that using what we've learned so far about Sequence to Sequence models. This notebook was updated to work with TensorFlow 1.1 and builds on the work of Dave Currie. Check out Dave's post [Text Summarization with Amazon Reviews](https://medium.com/towards-data-science/text-summarization-with-amazon-reviews-41801c2210b).

<img src="images/sequence-to-sequence.jpg"/>


## Dataset 

The dataset lives in the /data/ folder. At the moment, it is made up of the following files:
 * **letters_source.txt**: The list of input letter sequences. Each sequence is its own line. 
 * **letters_target.txt**: The list of target sequences we'll use in the training process. Each sequence here is a response to the input sequence in letters_source.txt with the same line number.

In [ ]:
import numpy as np
import time

import helper

source_path = 'data/letters_source.txt'
target_path = 'data/letters_target.txt'

source_sentences = helper.load_data(source_path)
target_sentences = helper.load_data(target_path)

Let's start by examining the current state of the dataset. `source_sentences` contains the entire input sequence file as text delimited by newline symbols.

In [74]:
source_sentences[:50].split('\n')

['bsaqq',
 'npy',
 'lbwuj',
 'bqv',
 'kial',
 'tddam',
 'edxpjpg',
 'nspv',
 'huloz',
 '']

`target_sentences` contains the entire output sequence file as text delimited by newline symbols.  Each line corresponds to the line from `source_sentences`.  `target_sentences` contains a sorted characters of the line.

In [75]:
target_sentences[:50].split('\n')

['abqqs',
 'npy',
 'bjluw',
 'bqv',
 'aikl',
 'addmt',
 'degjppx',
 'npsv',
 'hlouz',
 '']

## Preprocess
To do anything useful with it, we'll need to turn the each string into a list of characters: 

<img src="images/source_and_target_arrays.png"/>

Then convert the characters to their int values as declared in our vocabulary:


In [76]:
def extract_character_vocab(data):
    special_words = ['<PAD>', '<UNK>', '<GO>',  '<EOS>']

    set_words = set([character for line in data.split('\n') for character in line])
    int_to_vocab = {word_i: word for word_i, word in enumerate(special_words + list(set_words))}
    vocab_to_int = {word: word_i for word_i, word in int_to_vocab.items()}

    return int_to_vocab, vocab_to_int

# Build int2letter and letter2int dicts
source_int_to_letter, source_letter_to_int = extract_character_vocab(source_sentences)
target_int_to_letter, target_letter_to_int = extract_character_vocab(target_sentences)

# Convert characters to ids
source_letter_ids = [[source_letter_to_int.get(letter, source_letter_to_int['<UNK>']) for letter in line] for line in source_sentences.split('\n')]
target_letter_ids = [[target_letter_to_int.get(letter, target_letter_to_int['<UNK>']) for letter in line] + [target_letter_to_int['<EOS>']] for line in target_sentences.split('\n')] 

print("Example source sequence")
print(source_letter_ids[:3])
print("\n")
print("Example target sequence")
print(target_letter_ids[:3])

Example source sequence
[[7, 10, 25, 12, 12], [20, 13, 14], [18, 7, 16, 9, 4]]


Example target sequence
[[25, 7, 12, 12, 10], [20, 13, 14], [7, 4, 18, 9, 16]]


The last step in the preprocessing stage is to determine the the longest sequence size in the dataset we'll be using, then pad all the sequences to that length.

In [5]:
def pad_id_sequences(source_ids, source_letter_to_int, target_ids, target_letter_to_int, sequence_length):
    new_source_ids = [sentence + [source_letter_to_int['<pad>']] * (sequence_length - len(sentence)) \
                      for sentence in source_ids]
    new_target_ids = [sentence + [target_letter_to_int['<pad>']] * (sequence_length - len(sentence)) \
                      for sentence in target_ids]

    return new_source_ids, new_target_ids



This is the final shape we need them to be in. We can now proceed to building the model.

Sequence Length
7


Input sequence example
[[7, 10, 25, 12, 12, 0, 0], [20, 13, 14, 0, 0, 0, 0], [18, 7, 16, 9, 4, 0, 0]]


Target sequence example
[[25, 7, 12, 12, 10, 0, 0], [20, 13, 14, 0, 0, 0, 0], [7, 4, 18, 9, 16, 0, 0]]


## Model
#### Check the Version of TensorFlow
This will check to make sure you have the correct version of TensorFlow

In [77]:
from distutils.version import LooseVersion
import tensorflow as tf
from tensorflow.python.layers.core import Dense


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.0.1


### Hyperparameters

In [ ]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

### Input

In [79]:
def get_model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length


### Sequence to Sequence Model

We can now start defining the functions that will build the seq2seq model. We are building it from the bottom up with the following components:

    2.1 Encoder
        - Embedding
        - Encoder cell
    2.2 Decoder
        1- Process decoder inputs
        2- Set up the decoder
            - Embedding
            - Decoder cell
            - Dense output layer
            - Training decoder
            - Inference decoder
    2.3 Seq2seq model connecting the encoder and decoder
    2.4 Build the training graph hooking up the model with the 
        optimizer



### 2.1 Encoder

The first bit of the model we'll build is the encoder. Here, we'll embed the input data, construct our encoder, then pass the embedded data to the encoder.

- Embed the input data using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
<img src="images/embed_sequence.png" />

- Pass the embedded input into a stack of RNNs.  Save the RNN state and ignore the output.
<img src="images/encoder.png" />

In [ ]:
def encoding_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):


    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

## 2.2 Decoder

The decoder is probably the most involved part of this model. The following steps are needed to create it:

    1- Process decoder inputs
    2- Set up the decoder components
        - Embedding
        - Decoder cell
        - Dense output layer
        - Training decoder
        - Inference decoder


### Process Decoder Input


In the training process, the target sequences will be used in two different places:

 1. Using them to calculate the loss
 2. Feeding them to the decoder during training to make the model more robust.

Now we need to address the second point. Let's assume our targets look like this in their letter/word form (we're doing this for readibility. At this point in the code, these sequences would be in int form):


<img src="images/targets_1.png"/>

We need to do a simple transformation on the tensor before feeding it to the decoder:

1- We will feed an item of the sequence to the decoder at each time step. Think about the last timestep -- where the decoder outputs the final word in its output. The input to that step is the item before last from the target sequence. The decoder has no use for the last item in the target sequence in this scenario. So we'll need to remove the last item. 

We do that using tensorflow's tf.strided_slice() method. We hand it the tensor, and the index of where to start and where to end the cutting.

<img src="images/strided_slice_1.png"/>

2- The first item in each sequence we feed to the decoder has to be GO symbol. So We'll add that to the beginning.


<img src="images/targets_add_go.png"/>


Now the tensor is ready to be fed to the decoder. It looks like this (if we convert from ints to letters/symbols):

<img src="images/targets_after_processing_1.png"/>

In [90]:
# Process the input we'll feed to the decoder
def process_decoder_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input


### Set up the decoder components

        - Embedding
        - Decoder cell
        - Dense output layer
        - Training decoder
        - Inference decoder

#### 1- Embedding
Now that we have prepared the inputs to the training decoder, we need to embed them so they can be ready to be passed to the decoder. 

We'll create an embedding matrix like the following then have tf.nn.embedding_lookup convert our input to its embedded equivalent:
<img src="images/embeddings.png" />

#### 2- Decoder Cell
Then we declare our decoder cell. Just like the encoder, we'll use an tf.contrib.rnn.LSTMCell here as well.

We need to declare a decoder for the training process, and a decoder for the inference/prediction process. These two decoders will share their parameters (so that all the weights and biases that are set during the training phase can be used when we deploy the model).

First, we'll need to define the type of cell we'll be using for our decoder RNNs. We opted for LSTM.

#### 3- Dense output layer
Before we move to declaring our decoders, we'll need to create the output layer, which will be a tensorflow.python.layers.core.Dense layer that translates the outputs of the decoder to logits that tell us which element of the decoder vocabulary the decoder is choosing to output at each time step.

#### 4- Training decoder
Essentially, we'll be creating two decoders which share their parameters. One for training and one for inference. The two are similar in that both created using tf.contrib.seq2seq.**BasicDecoder** and tf.contrib.seq2seq.**dynamic_decode**. They differ, however, in that we feed the the target sequences as inputs to the training decoder at each time step to make it more robust.

We can think of the training decoder as looking like this (except that it works with sequences in batches):
<img src="images/sequence-to-sequence-training-decoder.png"/>

The training decoder **does not** feed the output of each time step to the next. Rather, the inputs to the decoder time steps are the target sequence from the training dataset (the orange letters).

#### 5- Inference decoder
The inference decoder is the one we'll use when we deploy our model to the wild.

<img src="images/sequence-to-sequence-inference-decoder.png"/>

We'll hand our encoder hidden state to both the training and inference decoders and have it process its output. TensorFlow handles most of the logic for us. We just have to use the appropriate methods from tf.contrib.seq2seq and supply them with the appropriate inputs.


In [ ]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, enc_state, dec_input):
    # 1. Decoder Embedding
    target_vocab_size = len(target_letter_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):

        # Helper for the training process. Used by BasicDecoder to read inputs.
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
        
        # Basic decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           enc_state,
                                                           output_layer) 
        
        # Perform dynamic decoding using the decoder
        training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)[0]
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                target_letter_to_int['<EOS>'])

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        enc_state,
                                                        output_layer)
        
        # Perform dynamic decoding using the decoder
        inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
         

    
    return training_decoder_output, inference_decoder_output

## 2.3 Seq2seq model 
Let's now go a step above, and hook up the encoder and decoder using the methods we just declared

In [100]:

def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, 
                  rnn_size, num_layers):
    
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(targets, target_letter_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(target_letter_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       enc_state, 
                                                                       dec_input) 
    
    return training_decoder_output, inference_decoder_output
    



Model outputs *training_decoder_output* and *inference_decoder_output* both contain a 'rnn_output' logits tensor that looks like this:

<img src="images/logits.png"/>

The logits we get from the training tensor we'll pass to tf.contrib.seq2seq.**sequence_loss()** to calculate the loss and ultimately the gradient.




In [101]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_model_inputs()
    
    # Create the training and inference logits
    training_decoder_output, inference_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_letter_to_int),
                                                                      len(target_letter_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    inference_logits = tf.identity(inference_decoder_output.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


## Get Batches

There's little processing involved when we retreive the batches. This is a simple example assuming batch_size = 2

Source sequences (it's actually in int form, we're showing the characters for clarity):

<img src="images/source_batch.png" />

Target sequences (also in int, but showing letters for clarity):

<img src="images/target_batch.png" />

In [ ]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def get_batches(targets, sources, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))
        
        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))
        
        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))
        
        yield pad_targets_batch, pad_sources_batch, pad_targets_lengths, pad_source_lengths

## Train
We're now ready to train our model. If you run into OOM (out of memory) issues during training, try to decrease the batch_size.

In [ ]:
# Split data to training and validation sets
train_source = source_letter_ids[batch_size:]
train_target = target_letter_ids[batch_size:]
valid_source = source_letter_ids[:batch_size]
valid_target = target_letter_ids[:batch_size]
(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 20 # Check training loss after every 20 batches

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(train_target, train_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>'])):
            
            # Training step
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            # Debug message updating us on the status of the training
            if batch_i % display_step == 0 and batch_i > 0:
                
                # Calculate validation cost
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))

    
    
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

Epoch   0 Batch    0/78 - Train Accuracy:  0.028, Validation Accuracy:  0.023, Loss:  3.419
Epoch   0 Batch    1/78 - Train Accuracy:  0.114, Validation Accuracy:  0.123, Loss:  3.396
Epoch   0 Batch    2/78 - Train Accuracy:  0.461, Validation Accuracy:  0.451, Loss:  3.369
Epoch   0 Batch    3/78 - Train Accuracy:  0.450, Validation Accuracy:  0.449, Loss:  3.345
Epoch   0 Batch    4/78 - Train Accuracy:  0.446, Validation Accuracy:  0.449, Loss:  3.323
Epoch   0 Batch    5/78 - Train Accuracy:  0.445, Validation Accuracy:  0.449, Loss:  3.296
Epoch   0 Batch    6/78 - Train Accuracy:  0.432, Validation Accuracy:  0.449, Loss:  3.268
Epoch   0 Batch    7/78 - Train Accuracy:  0.432, Validation Accuracy:  0.449, Loss:  3.236
Epoch   0 Batch    8/78 - Train Accuracy:  0.417, Validation Accuracy:  0.449, Loss:  3.208
Epoch   0 Batch    9/78 - Train Accuracy:  0.451, Validation Accuracy:  0.449, Loss:  3.150
Epoch   0 Batch   10/78 - Train Accuracy:  0.402, Validation Accuracy:  0.449, L

Epoch   1 Batch   17/78 - Train Accuracy:  0.491, Validation Accuracy:  0.474, Loss:  1.808
Epoch   1 Batch   18/78 - Train Accuracy:  0.443, Validation Accuracy:  0.473, Loss:  1.944
Epoch   1 Batch   19/78 - Train Accuracy:  0.481, Validation Accuracy:  0.473, Loss:  1.857
Epoch   1 Batch   20/78 - Train Accuracy:  0.502, Validation Accuracy:  0.474, Loss:  1.750
Epoch   1 Batch   21/78 - Train Accuracy:  0.429, Validation Accuracy:  0.473, Loss:  1.988
Epoch   1 Batch   22/78 - Train Accuracy:  0.451, Validation Accuracy:  0.474, Loss:  1.887
Epoch   1 Batch   23/78 - Train Accuracy:  0.494, Validation Accuracy:  0.475, Loss:  1.750
Epoch   1 Batch   24/78 - Train Accuracy:  0.465, Validation Accuracy:  0.480, Loss:  1.855
Epoch   1 Batch   25/78 - Train Accuracy:  0.450, Validation Accuracy:  0.478, Loss:  1.891
Epoch   1 Batch   26/78 - Train Accuracy:  0.439, Validation Accuracy:  0.479, Loss:  1.960
Epoch   1 Batch   27/78 - Train Accuracy:  0.458, Validation Accuracy:  0.478, L

Epoch   2 Batch   32/78 - Train Accuracy:  0.575, Validation Accuracy:  0.586, Loss:  1.350
Epoch   2 Batch   33/78 - Train Accuracy:  0.555, Validation Accuracy:  0.588, Loss:  1.403
Epoch   2 Batch   34/78 - Train Accuracy:  0.580, Validation Accuracy:  0.583, Loss:  1.322
Epoch   2 Batch   35/78 - Train Accuracy:  0.560, Validation Accuracy:  0.593, Loss:  1.356
Epoch   2 Batch   36/78 - Train Accuracy:  0.581, Validation Accuracy:  0.605, Loss:  1.380
Epoch   2 Batch   37/78 - Train Accuracy:  0.558, Validation Accuracy:  0.597, Loss:  1.370
Epoch   2 Batch   38/78 - Train Accuracy:  0.567, Validation Accuracy:  0.599, Loss:  1.340
Epoch   2 Batch   39/78 - Train Accuracy:  0.565, Validation Accuracy:  0.603, Loss:  1.380
Epoch   2 Batch   40/78 - Train Accuracy:  0.593, Validation Accuracy:  0.607, Loss:  1.334
Epoch   2 Batch   41/78 - Train Accuracy:  0.568, Validation Accuracy:  0.608, Loss:  1.367
Epoch   2 Batch   42/78 - Train Accuracy:  0.594, Validation Accuracy:  0.609, L

Epoch   3 Batch   49/78 - Train Accuracy:  0.663, Validation Accuracy:  0.710, Loss:  1.063
Epoch   3 Batch   50/78 - Train Accuracy:  0.652, Validation Accuracy:  0.717, Loss:  1.152
Epoch   3 Batch   51/78 - Train Accuracy:  0.686, Validation Accuracy:  0.711, Loss:  0.998
Epoch   3 Batch   52/78 - Train Accuracy:  0.674, Validation Accuracy:  0.709, Loss:  1.007
Epoch   3 Batch   53/78 - Train Accuracy:  0.685, Validation Accuracy:  0.718, Loss:  1.029
Epoch   3 Batch   54/78 - Train Accuracy:  0.671, Validation Accuracy:  0.714, Loss:  1.049
Epoch   3 Batch   55/78 - Train Accuracy:  0.657, Validation Accuracy:  0.706, Loss:  1.071
Epoch   3 Batch   56/78 - Train Accuracy:  0.685, Validation Accuracy:  0.715, Loss:  1.018
Epoch   3 Batch   57/78 - Train Accuracy:  0.664, Validation Accuracy:  0.717, Loss:  1.074
Epoch   3 Batch   58/78 - Train Accuracy:  0.673, Validation Accuracy:  0.720, Loss:  1.062
Epoch   3 Batch   59/78 - Train Accuracy:  0.662, Validation Accuracy:  0.717, L

Epoch   4 Batch   63/78 - Train Accuracy:  0.777, Validation Accuracy:  0.768, Loss:  0.771
Epoch   4 Batch   64/78 - Train Accuracy:  0.743, Validation Accuracy:  0.767, Loss:  0.818
Epoch   4 Batch   65/78 - Train Accuracy:  0.771, Validation Accuracy:  0.766, Loss:  0.775
Epoch   4 Batch   66/78 - Train Accuracy:  0.781, Validation Accuracy:  0.768, Loss:  0.755
Epoch   4 Batch   67/78 - Train Accuracy:  0.758, Validation Accuracy:  0.766, Loss:  0.782
Epoch   4 Batch   68/78 - Train Accuracy:  0.746, Validation Accuracy:  0.766, Loss:  0.832
Epoch   4 Batch   69/78 - Train Accuracy:  0.792, Validation Accuracy:  0.772, Loss:  0.688
Epoch   4 Batch   70/78 - Train Accuracy:  0.735, Validation Accuracy:  0.773, Loss:  0.823
Epoch   4 Batch   71/78 - Train Accuracy:  0.768, Validation Accuracy:  0.771, Loss:  0.755
Epoch   4 Batch   72/78 - Train Accuracy:  0.742, Validation Accuracy:  0.772, Loss:  0.838
Epoch   4 Batch   73/78 - Train Accuracy:  0.713, Validation Accuracy:  0.772, L

...

Epoch  54 Batch   73/78 - Train Accuracy:  1.000, Validation Accuracy:  0.989, Loss:  0.007
Epoch  54 Batch   74/78 - Train Accuracy:  1.000, Validation Accuracy:  0.993, Loss:  0.005
Epoch  54 Batch   75/78 - Train Accuracy:  1.000, Validation Accuracy:  0.993, Loss:  0.005
Epoch  54 Batch   76/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  55 Batch    0/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.006
Epoch  55 Batch    1/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  55 Batch    2/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  55 Batch    3/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  55 Batch    4/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  55 Batch    5/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, Loss:  0.005
Epoch  55 Batch    6/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, L

Epoch  56 Batch   13/78 - Train Accuracy:  1.000, Validation Accuracy:  0.996, Loss:  0.005
Epoch  56 Batch   14/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.004
Epoch  56 Batch   15/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  56 Batch   16/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  56 Batch   17/78 - Train Accuracy:  1.000, Validation Accuracy:  0.990, Loss:  0.004
Epoch  56 Batch   18/78 - Train Accuracy:  1.000, Validation Accuracy:  0.990, Loss:  0.005
Epoch  56 Batch   19/78 - Train Accuracy:  1.000, Validation Accuracy:  0.990, Loss:  0.005
Epoch  56 Batch   20/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.003
Epoch  56 Batch   21/78 - Train Accuracy:  1.000, Validation Accuracy:  0.993, Loss:  0.005
Epoch  56 Batch   22/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, Loss:  0.005
Epoch  56 Batch   23/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, L

Epoch  57 Batch   28/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, Loss:  0.005
Epoch  57 Batch   29/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, Loss:  0.005
Epoch  57 Batch   30/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, Loss:  0.005
Epoch  57 Batch   31/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  57 Batch   32/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  57 Batch   33/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  57 Batch   34/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.004
Epoch  57 Batch   35/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  57 Batch   36/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.004
Epoch  57 Batch   37/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  57 Batch   38/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, L

Epoch  58 Batch   45/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.005
Epoch  58 Batch   46/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.004
Epoch  58 Batch   47/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.003
Epoch  58 Batch   48/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.003
Epoch  58 Batch   49/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  58 Batch   50/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  58 Batch   51/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  58 Batch   52/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  58 Batch   53/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  58 Batch   54/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  58 Batch   55/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, L

Epoch  59 Batch   59/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.006
Epoch  59 Batch   60/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.004
Epoch  59 Batch   61/78 - Train Accuracy:  0.996, Validation Accuracy:  0.992, Loss:  0.006
Epoch  59 Batch   62/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.003
Epoch  59 Batch   63/78 - Train Accuracy:  1.000, Validation Accuracy:  0.992, Loss:  0.003
Epoch  59 Batch   64/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.005
Epoch  59 Batch   65/78 - Train Accuracy:  1.000, Validation Accuracy:  0.990, Loss:  0.005
Epoch  59 Batch   66/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  59 Batch   67/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.003
Epoch  59 Batch   68/78 - Train Accuracy:  1.000, Validation Accuracy:  0.991, Loss:  0.004
Epoch  59 Batch   69/78 - Train Accuracy:  1.000, Validation Accuracy:  0.994, L

## Prediction

In [20]:
input_sentence = 'fjeixg'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(text)]*batch_size, 
                                      source_sequence_length: [len(text)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nSource')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

Input
  Word Ids:      [26, 4, 5, 23, 28, 27, 0]
  Input Words: ['f', 'j', 'e', 'i', 'x', 'g', '<pad>']

Prediction
  Word Ids:      [5, 26, 27, 23, 4, 28, 0]
  Chatbot Answer Words: ['e', 'f', 'g', 'i', 'j', 'x', '<pad>']
